# Hypothesis Testing in Healthcare: Drug Safety

A pharmaceutical company GlobalXYZ has just completed a randomized controlled drug trial. To promote transparency and reproducibility of the drug's outcome, they (GlobalXYZ) have presented the dataset to your organization, a non-profit that focuses primarily on drug safety.

The dataset provided contained five adverse effects, demographic data, vital signs, etc. Your organization is primarily interested in the drug's adverse reactions. It wants to know if the adverse reactions, if any, are of significant proportions. It has asked you to explore and answer some questions from the data.

The dataset `drug_safety.csv` was obtained from [Hbiostat](https://hbiostat.org/data/) courtesy of the Vanderbilt University Department of Biostatistics. It contained five adverse effects: headache, abdominal pain, dyspepsia, upper respiratory infection, chronic obstructive airway disease (COAD), demographic data, vital signs, lab measures, etc. The ratio of drug observations to placebo observations is 2 to 1.

For this project, the dataset has been modified to reflect the presence and absence of adverse effects `adverse_effects` and the number of adverse effects in a single individual `num_effects`.

The columns in the modified dataset are: 

| Column | Description |
|--------|-------------|
|`sex` | The gender of the individual |
|`age` | The age of the individual |
|`week` | The week of the drug testing |
|`trx` | The treatment (Drug) and control (Placebo) groups | 
|`wbc` | The count of white blood cells |
|`rbc` | The count of red blood cells |
|`adverse_effects` | The presence of at least a single adverse effect |
|`num_effects` | The number of adverse effects experienced by a single individual |

The original dataset can be found [here](https://hbiostat.org/data/repo/safety.rda).

Your organization has asked you to explore and answer some questions from the data collected. See the project instructions.

In [36]:
# Import packages
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import pingouin
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
drug_safety = pd.read_csv("drug_safety.csv")

# Start coding here...


In [37]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind
from statsmodels.stats.proportion import proportions_ztest

# --------------------------------
# 1. Drug vs Placebo 이상반응 비율 차이 검정
# --------------------------------

# 치료군 × 이상반응 여부 교차표
adverse_table = pd.crosstab(
    drug_safety["trx"],
    drug_safety["adverse_effects"]
)

# 이상반응 발생한 개수
count = adverse_table.iloc[:, 1].values

# 각 그룹의 전체 표본 수
nobs = adverse_table.sum(axis=1).values

# 두 표본 비율 검정
_, two_sample_p_value = proportions_ztest(
    count,
    nobs
)


# --------------------------------
# 2. 이상반응 개수와 그룹의 독립성 검정
# --------------------------------

num_effects_table = pd.crosstab(
    drug_safety["trx"],
    drug_safety["num_effects"]
)

_, num_effects_p_value, _, _ = chi2_contingency(num_effects_table)


# --------------------------------
# 3. Drug vs Placebo 나이 차이 검정
# --------------------------------

drug_ages = drug_safety.loc[drug_safety["trx"] == "Drug", "age"]
placebo_ages = drug_safety.loc[drug_safety["trx"] == "Placebo", "age"]

_, age_group_effects_p_value = ttest_ind(
    drug_ages,
    placebo_ages,
    equal_var=False
)

from scipy.stats import mannwhitneyu

_, age_group_effects_p_value = mannwhitneyu(
    drug_ages,
    placebo_ages,
    alternative="two-sided"
)

print("two_sample_p_value:", two_sample_p_value)
print("num_effects_p_value:", num_effects_p_value)
print("age_group_effects_p_value:", age_group_effects_p_value)

two_sample_p_value: 0.9639333330262475
num_effects_p_value: 0.6150123339426765
age_group_effects_p_value: 0.25696267004066287
